In this project, I have analyzed the OpenStreetMap data for the [San Francisco](http://www.openstreetmap.org/relation/111968) city area. I have broken down the analysis into four parts-

* __Data Extraction__
* __Data Cleaning and transformation__
* __Uploading to database__
* __Clustering Analysis__

__Data Extraction:__ I have downloaded the OSM XML file from OpenStreetMap for the San Francisco city area. Since the file is 82 MB in size, I have created a smaller sample file for looking at the data and get an idea of what fields need to be cleaned. 

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET    # For parsing XML 

k = 20   # Parameter: take every k-th top level element

def get_element(input_filename):
    tags = ('node', 'way', 'relation')
    # Yield element if it is the right type of tag    
    context = iter(ET.iterparse(input_filename, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def create_sample(input_filename, output_sample_filename):
    with open(output_sample_filename, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')
        # Write every kth top level element
        for i, element in enumerate(get_element(input_filename)):
            if i % k == 0:
                output.write(ET.tostring(element, encoding='utf-8'))
        output.write('</osm>')

create_sample('san-francisco.osm', 'sample_sf.osm')

_Structure of the XML file_ - It is a list of instances of data primitives (nodes, ways, and relations).
* _Node_ - A single point in space defined by its latitude, longitude and node id. Nodes can be used to define point features, and will have one or several tags to define the typ of feature. For example in this case, nodes are used to define restaurants and will have several tags for cuisine type, name, etc. 
* _Way_ - A way is an ordered list of nodes which normally also has at least one tag.
* _Relation_ - It is a logical or geographic relationships between objects.


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Use iterative parsing to process the map file to find out what and how many tags are there
# Get an idea on how much of which data you can expect to have in the map

import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
    # Returns a dictionary with the tag name as the key and number of times this tag can be encountered in the map as value.
    tag_count = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tag_count:
            tag_count[elem.tag] = 0
        tag_count[elem.tag] = tag_count[elem.tag] + 1   
    return tag_count
        
pprint.pprint(count_tags('sample_sf.osm'))

{'member': 3118,
 'nd': 362013,
 'node': 307268,
 'osm': 1,
 'relation': 300,
 'tag': 98594,
 'way': 37584}


In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

def get_user(element):
    userid = element.get("uid")
    return userid

def process_users(filename):    # return a set of unique user IDs ("uid")
    users = set()
    for _, element in ET.iterparse(filename):
        uid = "uid"
        if uid in element.keys():
            userid = get_user(element)
            users.add(userid)
    return users

print len(process_users('sample_sf.osm'))

1275


In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint
import re
"""
Check the "k" value for each "<tag>" and see if there are any potential problems.
Expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with problematic characters.

  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
"""

# Three regular expressions to check for certain patterns in the tags
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    element_tag = element.tag
    if element_tag == "tag":
        k_attribute_value = element.get("k")
        if lower.match(k_attribute_value):
            keys["lower"] += 1
        elif lower_colon.match(k_attribute_value):
            keys["lower_colon"] +=1
        elif problemchars.search(k_attribute_value):
            keys["problemchars"] +=1
        else:
            keys["other"] +=1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

print (process_map('sample_sf.osm'))

{'problemchars': 10, 'lower': 63014, 'other': 1305, 'lower_colon': 34265}


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.   
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "sample_sf.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Circle", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace", "Walk", "Way"]

mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Rd": "Road",
            "Ave": "Avenue",
            "Dr": "Drive"
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    words = name.split()
    for i in range(len(words)):
        if words[i] in mapping:
            words[i] = mapping[words[i]]  
    return ' '.join(words)

st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))

{'1': set(['W Of Us 101 @ Jct Sr 1']),
 '105': set(['Grand Avenue #105', 'N California Blvd #105']),
 '3': set(['University Dr. Suite 3']),
 '39': set(['Pier 39']),
 '4.5': set(['SF 80 PM 4.5']),
 'A': set(['Pier 50 A']),
 'Alameda': set(['Alameda', 'The Alameda']),
 'Alley': set(['August Alley',
               'Hodges Alley',
               'Redfield Alley',
               'Ross Alley']),
 'Ave': set(['Esplanade Ave',
             'Lorton Ave',
             'Magnolia Ave',
             'Pennsylvania Ave',
             'Tehama Ave',
             'Telegraph Ave']),
 'Ave.': set(['Edes Ave.', 'Willie Stargell Ave.']),
 'Bay': set(['Bay']),
 'Blvd': set(['Newark Blvd',
              'Under Ramp Sw Quad Of Us 101 / Sr 92 Ic Off 19th Ave & Fashion Island Blvd']),
 'Bridgeway': set(['Bridgeway']),
 'Broadway': set(['Broadway']),
 'Center': set(['Bon Air Center', 'South Shore Center', 'Westlake Center']),
 'Circle': set(['Blossom Circle',
                'Citrus Circle',
                'Colu

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so I will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files. These csv files can then easily be imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Write each data structure to the appropriate .csv files

In the next piece of code I am loading the data, performing iterative parsing and writing the
output to csv files. The shape_element function will transform each element into the correct format. To make this process easier 
__Shape Element Function__
The function will take as input an iterparse Element object and return a dictionary.

__ If the element top level tag is "node":__
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,
- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

__ If the element top level tag is "way":__
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
- user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}

To summarize, the columns in the nodes table will be:

id    user    uid    version    lat    lon    timestamp    changeset

Columns in the nodes_tags table will be: 

id    key    value    type

In [7]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')  
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema   

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def is_street_name(full_key_name):
    return (full_key_name == "addr:street")

def update_street_name(name):
    mapping = { "St": "Street", "St.": "Street", "Rd.": "Road", "Rd": "Road", "Ave": "Avenue", "Dr": "Drive"}
    words = name.split()
    for i in range(len(words)):
        if words[i] in mapping:
            words[i] = mapping[words[i]]  
    return ' '.join(words)

def clean_value(value, key_name, full_key_name):
    if is_street_name(full_key_name):
        return update_street_name(value)
    elif 'cuisine' in key_name.lower():
        return value.replace('_', ' ').lower()
    else:
        return value

# ================================================== #
#       Application Specific Helper Functions        #
# ================================================== #
def parse_list(tag, parent_id, default_tag_type):
    # Parse and clean tags. 
    all_rows = []
    if PROBLEMCHARS.search(tag.get('k')):
        return all_rows
    else:
        values = tag.get('v').split(';') # example values = "brunch;breakfast;coffee"
        for value in values:
            parsed_tag = {}
            parsed_tag['id'] = parent_id
            k_value = tag.get('k')  # example "addr:street"  "diet:vegan"
            idx_colon = k_value.find(':') 
            if idx_colon == -1: # example = "cuisine"
                # Parse for k without ':'
                parsed_tag['type'] = default_tag_type
                parsed_tag['key'] = k_value
            else:
                parsed_tag['type'] = k_value[0:idx_colon]  # example "addr:street"
                parsed_tag['key'] = k_value[idx_colon+1:]
            parsed_tag['value'] = clean_value(value, parsed_tag['key'], k_value)
            all_rows.append(parsed_tag)
        return all_rows

def parse_node(element, default_tag_type):
    # Node parsing logic here. Returns node_attributes and node_tags.
    node_attributes = { key:element.get(key) for key in NODE_FIELDS }
    tags = []
    for child in element.findall('tag'):
        tag = parse_list(child, element.get('id'), default_tag_type)
        if tag: 
            tags.extend(tag)
    return node_attributes, tags
    
def parse_way_node(element, parent_id, idx):
    return {'id':parent_id, 'node_id': element.get('ref'), 'position':idx}
    
def parse_way(element, default_tag_type):
    # Way parsing logic here. Returns way_attributes, way_nodes, and way_tags.
    way_attributes = { key:element.get(key) for key in WAY_FIELDS }
    way_nodes = []
    for idx, child in enumerate(element.findall('nd')):
        way_nodes.append(parse_way_node(child, element.get('id'), idx))
    tags = []
    for child in element.findall('tag'):
        tag = parse_list(child, element.get('id'), default_tag_type)
        if tag:
            tags.extend(tag)
    return way_attributes, way_nodes, tags

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    # Clean and shape node or way XML element to Python dict
    if element.tag == 'node':
        node_attribs, tags = parse_node(element, default_tag_type)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        way_attribs, way_nodes, tags = parse_way(element, default_tag_type)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })
    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    NODES_PATH = "nodes.csv"
    NODE_TAGS_PATH = "nodes_tags.csv"
    WAYS_PATH = "ways.csv"
    WAY_NODES_PATH = "ways_nodes.csv"
    WAY_TAGS_PATH = "ways_tags.csv"

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

process_map("san-francisco.osm")

While uploading to database, I still found 5 rows with empty uid and username cells in the nodes csv file. So I went back and removed the rows which had empty uid cells. 


In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import unicodecsv
import pandas as pd

all_data = pd.read_csv('nodes.csv')
print(pd.isnull(all_data).any(1).nonzero()[0])

def read_csv(filename, outputfile):
    rows_to_remove = [53800, 53801, 53804, 53835, 53846]
    with open(filename, 'rb') as f_read:
        reader = unicodecsv.DictReader(f_read)
        with open(outputfile, 'wb') as f_write:
            writer = unicodecsv.DictWriter(f_write, reader.fieldnames)
            writer.writeheader()
            for row_idx, row in enumerate(reader):
                if row_idx not in rows_to_remove:
                    writer.writerow(row)

read_csv('nodes.csv', 'nodes-cleaned.csv')

all_data = pd.read_csv('nodes-cleaned.csv')
print(pd.isnull(all_data).any(1).nonzero()[0])

[53800 53801 53804 53835 53846]
[]
